In [123]:
from pxr import Usd,Sdf

In [124]:
stage = Usd.Stage.CreateInMemory()
prim = stage.DefinePrim("/VariantSet")

In [125]:
vset = prim.GetVariantSets().AddVariantSet('hogehoge')
vset.AddVariant('A')
vset.AddVariant('B')
vset.AddVariant('C')
print(vset)

In [126]:
# VariantSetを取得
prim.GetVariantSets().GetNames()

['hogehoge']

In [127]:
vset = prim.GetVariantSets().GetVariantSet("hogehoge")
vset.SetVariantSelection('A')
# 今選択されているものを表示
print(vset.GetVariantSelection())
# VariatnSetがあるPrimを取得
print(vset.GetPrim())

A
Usd.Prim(</VariantSet>)


In [128]:
vset.SetVariantSelection('A')
with vset.GetVariantEditContext():
    # VariantSet「A」を選んでいる時には VariantSet/hoge というPrimができあがる
    childPrim = stage.DefinePrim(prim.GetPath().AppendChild("hoge"))

In [129]:
vset.SetVariantSelection('B')
with vset.GetVariantEditContext():
    childPrim = stage.DefinePrim(prim.GetPath().AppendChild("hogeB"))
    childPrim.GetReferences().AddReference(r"D:\work\usd_py36\usd\layerB.usda")

In [130]:
vset.SetVariantSelection('C')
with vset.GetVariantEditContext():
    prim.CreateAttribute("TEST",Sdf.ValueTypeNames.String).Set("HOGE")


In [131]:
print(stage.GetRootLayer().ExportToString())

#usda 1.0

def "VariantSet" (
    variants = {
        string hogehoge = "C"
    }
    prepend variantSets = "hogehoge"
)
{
    variantSet "hogehoge" = {
        "A" {
            def "hoge"
            {
            }

        }
        "B" {
            def "hogeB" (
                prepend references = @D:\work\usd_py36\usd\layerB.usda@
            )
            {
            }

        }
        "C" {
            custom string TEST = "HOGE"

        }
    }
}




In [132]:
vset.SetVariantSelection('A')
print(stage.Flatten().ExportToString())

#usda 1.0
(
    doc = """Generated from Composed Stage of root layer 
"""
)

def "VariantSet"
{
    def "hoge"
    {
    }
}




In [133]:
vset.SetVariantSelection('B')
print(stage.Flatten().ExportToString())

#usda 1.0
(
    doc = """Generated from Composed Stage of root layer 
"""
)

def "VariantSet"
{
    def "hogeB"
    {
    }
}




In [134]:
vset.SetVariantSelection('C')
print(stage.Flatten().ExportToString())

#usda 1.0
(
    doc = """Generated from Composed Stage of root layer 
"""
)

def "VariantSet"
{
    custom string TEST = "HOGE"
}




In [135]:
stage.GetRootLayer().Export("D:/test.usda")

True

In [136]:
prim.GetPrimStack()

[Sdf.Find('anon:000002C06682DA70:tmp.usda', '/VariantSet'),
 Sdf.Find('anon:000002C06682DA70:tmp.usda', '/VariantSet{hogehoge=C}')]

In [138]:
vset.SetVariantSelection('B')
a = stage.GetPrimAtPath("/VariantSet/hogeB")
print(a.GetPrimStack())

[Sdf.Find('anon:000002C06682DA70:tmp.usda', '/VariantSet{hogehoge=B}hogeB')]


In [139]:
spec = a.GetPrimStack()[0] #PrimSpecを取得できる

In [140]:
# PrimSpecからReferenceのUSDAは取得できるっぽい
for ref in spec.referenceList.prependedItems:
    print(ref.assetPath)

D:\work\usd_py36\usd\layerB.usda
